# Keras Tuner CNN

In [1]:
import keras_tuner
import tensorflow as tf
import numpy as np

# Load Data

In [2]:

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()


# Prepare the data, split, and reshape

In [3]:

# Reshape the images to have the channel dimension.
x_train = (x_train.reshape(x_train.shape + (1,)) / 255.0)[:1000]
y_train = y_train.astype(np.int64)[:1000]
x_test = (x_test.reshape(x_test.shape + (1,)) / 255.0)[:100]
y_test = y_test.astype(np.int64)[:100]


# Build the model

dynamic count of CONV Layers, with dynamic filters, kernel_size, optimizers

In [15]:
def build_model(hp):

    inputs = tf.keras.Input(shape=(28, 28, 1))
    x = inputs

    # Dynamic count of CONV Layers
    for i in range(hp.Int("conv_layers", 1, 3, default=3)):
        x = tf.keras.layers.Conv2D(

            filters=hp.Int("filters_" + str(i), 4, 32, step=4, default=8),
            kernel_size=hp.Int("kernel_size_" + str(i), 3, 5),
            activation="relu",
            padding="same",
        )(x) # (x): see the comment up

        if hp.Choice("pooling" + str(i), ["max", "avg"]) == "max":
            x = tf.keras.layers.MaxPooling2D()(x)
        else:
            x = tf.keras.layers.AveragePooling2D()(x)

        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.ReLU()(x)

    if hp.Choice("global_pooling", ["max", "avg"]) == "max":
        x = tf.keras.layers.GlobalMaxPooling2D()(x)
    else:
        x = tf.keras.layers.GlobalAveragePooling2D()(x)


    outputs = tf.keras.layers.Dense(10, activation="softmax")(x)

    model = tf.keras.Model(inputs, outputs)

    # dynamic optimizer choice
    optimizer = hp.Choice("optimizer", ["adam", "sgd"])
    model.compile(
        optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"]
    )
    return model

# Build the tuner

In [16]:
tuner = keras_tuner.Hyperband(
    hypermodel= build_model,
    objective="val_accuracy",
    max_epochs=2,
    factor=3,
    hyperband_iterations=1,
    distribution_strategy=tf.distribute.MirroredStrategy(),
    directory="keras_tuner",
    project_name="MNIST",
    overwrite=True,
)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


# Train the tuner

In [17]:
tuner.search(
    x_train,
    y_train,
    steps_per_epoch=600,
    validation_data=(x_test, y_test),
    validation_steps=100,
    callbacks=[tf.keras.callbacks.EarlyStopping("val_accuracy")],
)

Trial 2 Complete [00h 00m 12s]
val_accuracy: 0.699999988079071

Best val_accuracy So Far: 0.699999988079071
Total elapsed time: 00h 00m 20s


# Ge the result summary

In [18]:
tuner.results_summary()

Results summary
Results in keras_tuner\MNIST
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 0001 summary
Hyperparameters:
conv_layers: 3
filters_0: 24
kernel_size_0: 4
pooling0: max
filters_1: 20
kernel_size_1: 4
pooling1: avg
filters_2: 32
kernel_size_2: 4
pooling2: avg
global_pooling: avg
optimizer: sgd
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.699999988079071

Trial 0000 summary
Hyperparameters:
conv_layers: 1
filters_0: 28
kernel_size_0: 5
pooling0: max
filters_1: 28
kernel_size_1: 4
pooling1: max
filters_2: 32
kernel_size_2: 5
pooling2: avg
global_pooling: max
optimizer: sgd
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 0.4699999988079071


# Get the best Hyper parameter combination

In [20]:
best_hps  = tuner.get_best_hyperparameters() # get a list of top 1 hp

model = build_model(best_hps[0]) # [0]get the top best hp and build the model

 

model.fit(x=x_train, y=y_train, epochs= 1)


32/32 [==============================] - 2s 13ms/step - loss: 2.2026 - accuracy: 0.2090
